## Installere pakker


In [2]:
pip install -r requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'
Note: you may need to restart the kernel to use updated packages.




## Importering av pakker

In [3]:
import pyarrow as pa
import pyarrow.parquet as pq
import geopandas as gpd
from IPython.display import display
import os
import duckdb
import folium

## Konstante variabler

Partisjonering

In [10]:


# Opprett DuckDB-tilkobling
con = duckdb.connect()

# Les inn data fra en Parquet-fil
query = """
    SELECT *
    FROM read_parquet('/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/hais_2024-12-01.snappy.parquet.geo.parquet')
"""
data = con.execute(query).fetchdf()

# Lukk tilkoblingen
con.close()

# Konverter Pandas DataFrame til PyArrow Table
table = pa.Table.from_pandas(data)

# Definer output path
output_path = '/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/PartisjonertDataShipType'

# Skriv til partisjonerte Parquet-filer
pq.write_to_dataset(
    table,
    root_path=output_path,
    partition_cols=['ship_type']  # Partisjonering
)

print(f"Data er skrevet til {output_path} i partisjonerte Parquet-filer.")


Data er skrevet til /Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/PartisjonertDataShipType i partisjonerte Parquet-filer.


In [5]:
# Koble til DuckDB
conn = duckdb.connect()

# Angi filbanen til geoparquet-filen
file_path = '/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/hais_2024-12-01.snappy.parquet.geo.parquet'

# Les geoparquet-filen med DuckDB
query = f"""
SELECT *, EXTRACT(HOUR FROM date_time_utc) AS hour
FROM read_parquet('{file_path}')
"""

# Kjør spørringen og hent resultatene
df = conn.execute(query).fetchdf()

# Lagre partisjonerte filer (f.eks., én fil per time)
for hour in df['hour'].unique():
    partition_df = df[df['hour'] == hour]
    partition_df.to_parquet(f'partitioned_{hour}.parquet')

# Lukk tilkoblingen
conn.close()

In [6]:


# Koble til DuckDB
conn = duckdb.connect()

# Angi filbanen til geoparquet-filen
file_path = '/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/hais_2024-12-01.snappy.parquet.geo.parquet'

# Angi katalogen der de partisjonerte filene skal lagres
output_dir = '/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/PartisjonertDataTimer'

# Opprett katalogen hvis den ikke eksisterer
os.makedirs(output_dir, exist_ok=True)

# SQL-spørring for å hente dataene og partisjonere etter timer
query = f"""
SELECT *, EXTRACT(HOUR FROM date_time_utc) AS hour
FROM read_parquet('{file_path}')
"""

# Kjør spørringen og hent resultatene
df = conn.execute(query).fetchdf()

# Lagre partisjonerte filer (f.eks., én fil per time) i den angitte katalogen
for hour in df['hour'].unique():
    partition_df = df[df['hour'] == hour]

    # Bygg stien til den partisjonerte filen
    output_file = os.path.join(output_dir, f'partitioned_{hour}.parquet')

    # Lagre filen
    partition_df.to_parquet(output_file)

# Lukk tilkoblingen
conn.close()


Leser filen

In [14]:
import pandas as pd

df = pd.read_parquet("/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/PartisjonertDataTimer/partitioned_1.parquet")
print(df.head())


         date_time_utc       mmsi  longitude   latitude  status  \
47 2024-12-01 01:54:37  257565700   7.476913  58.020353       5   
48 2024-12-01 01:48:42  257565700   7.476913  58.020353       5   
49 2024-12-01 01:40:51  257565700   7.476915  58.020353       5   
50 2024-12-01 01:36:34  257565700   7.476915  58.020353       5   
51 2024-12-01 01:34:37  257565700   7.476917  58.020353       5   

    course_over_ground  speed_over_ground  rate_of_turn  maneuvre  imo  ...  \
47               195.5                0.0         -15.0       0.0    0  ...   
48               277.8                0.0           6.0       0.0    0  ...   
49                85.0                0.0          -9.0       0.0    0  ...   
50                88.6                0.0         -12.0       0.0    0  ...   
51               124.8                0.0          -7.0       0.0    0  ...   

          ship_name ship_type  length  draught  data_source ais_class  \
47  RESCUE 157 BILL        51      14      0.0   

Filtrering av rader - Setter en limit til kun 5 rader, den henter dermet kun de første 5 radene i filen og setter det i et kart


In [15]:

# Opprett DuckDB-tilkobling
con = duckdb.connect()

# Leser fra parquet-filen
query = """
    SELECT *
    FROM read_parquet('/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/hais_2024-12-01.snappy.parquet.geo.parquet')
"""

# Definer batch-størrelse til 100 rader
batch_size = 5
offset = 0

# Bygg spørring for å hente 5 rader
batch_query = f"{query} LIMIT {batch_size} OFFSET {offset}"
batch = con.execute(batch_query).fetchdf()

# Sjekk om vi har fått noen data
if len(batch) == 0:
    print("Ingen data funnet.")
else:
    print("Streamer 5 rader:")
    display(batch.head())  # Viser de første radene i en tabell

    # Hvis filen inneholder geodata, for eksempel med 'latitude' og 'longitude'-kolonner,
    # kan vi lage et enkelt Folium-kart:
    if "latitude" in batch.columns and "longitude" in batch.columns:
        avg_lat = batch["latitude"].mean()
        avg_lon = batch["longitude"].mean()
        m = folium.Map(location=[avg_lat, avg_lon], zoom_start=7)

        for _, row in batch.iterrows():
            lat = row["latitude"]
            lon = row["longitude"]
            if not pd.isna(lat) and not pd.isna(lon):
                folium.Marker(
                    location=[lat, lon],
                    popup=str(row.iloc[:5].to_dict())
                ).add_to(m)

        display(m)
    else:
        print("Ingen 'latitude'/'longitude'-kolonner funnet.")

# Lukk tilkoblingen
con.close()


Streamer 5 rader:


,date_time_utc,mmsi,longitude,latitude,status,course_over_ground,speed_over_ground,rate_of_turn,maneuvre,imo,callsign,ship_name,ship_type,length,draught,data_source,ais_class,hex_7,hex_14,geometry
0,2024-12-01 05:58:44,257565700,7.476903,58.020340,5,215.0,0.0,-9.0,0.0,0,LF5484,RESCUE 157 BILL,51,14,0.0,G,A,608154275335372799,639679472713910703,"[1, 1, 0, 0, 0, 97, 208, 182, 88, 89, 232, 29,..."
1,2024-12-01 05:30:48,257565700,7.476915,58.020353,5,3.2,0.0,16.0,0.0,0,LF5484,RESCUE 157 BILL,51,14,0.0,G,A,608154275335372799,639679472713910327,"[1, 1, 0, 0, 0, 42, 227, 223, 103, 92, 232, 29..."
2,2024-12-01 05:16:51,257565700,7.476912,58.020327,5,55.2,0.1,15.0,0.0,0,LF5484,RESCUE 157 BILL,51,14,0.0,G,A,608154275335372799,639679472713910663,"[1, 1, 0, 0, 0, 209, 31, 187, 135, 91, 232, 29..."
3,2024-12-01 05:12:51,257565700,7.476917,58.020340,5,134.2,0.0,16.0,0.0,0,LF5484,RESCUE 157 BILL,51,14,0.0,G,A,608154275335372799,639679472713910671,"[1, 1, 0, 0, 0, 95, 120, 70, 215, 92, 232, 29,..."
4,2024-12-01 05:06:46,257565700,7.476915,58.020327,5,26.6,0.0,10.0,0.0,0,LF5484,RESCUE 157 BILL,51,14,0.0,G,A,608154275335372799,639679472713910663,"[1, 1, 0, 0, 0, 42, 227, 223, 103, 92, 232, 29..."


""Filtrering av kolonner


In [14]:
# Opprett DuckDB-tilkobling
con = duckdb.connect()

# Velg spesifikke kolonner
query = """
    SELECT date_time_utc, geometry, ship_name
    FROM read_parquet('/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/hais_2024-12-01.snappy.parquet.geo.parquet')
"""

# Utfør spørringen
filtered_data = con.execute(query).fetchdf()

# Vis de første radene
display(filtered_data.head())

# Lukk tilkoblingen
con.close()


,date_time_utc,geometry,ship_name
0,2024-12-01 05:58:44,"[1, 1, 0, 0, 0, 97, 208, 182, 88, 89, 232, 29,...",RESCUE 157 BILL
1,2024-12-01 05:30:48,"[1, 1, 0, 0, 0, 42, 227, 223, 103, 92, 232, 29...",RESCUE 157 BILL
2,2024-12-01 05:16:51,"[1, 1, 0, 0, 0, 209, 31, 187, 135, 91, 232, 29...",RESCUE 157 BILL
3,2024-12-01 05:12:51,"[1, 1, 0, 0, 0, 95, 120, 70, 215, 92, 232, 29,...",RESCUE 157 BILL
4,2024-12-01 05:06:46,"[1, 1, 0, 0, 0, 42, 227, 223, 103, 92, 232, 29...",RESCUE 157 BILL


Filtrering av tidsstempler (og visualisering i kart)

In [16]:


# Opprett DuckDB-tilkobling
con = duckdb.connect()

# Definer tidsintervallet
start_time = "2024-12-01 05:12:51"
end_time = "2024-12-01 05:14:00"

# SQL-spørring for å filtrere data basert på tidspunkt
query = f"""
    SELECT *
    FROM read_parquet('/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/hais_2024-12-01.snappy.parquet.geo.parquet')
    WHERE CAST(date_time_utc AS TIMESTAMP)
    BETWEEN TIMESTAMP '{start_time}' AND TIMESTAMP '{end_time}'
"""

# Utfør spørringen og hent dataen
filtered_data = con.execute(query).fetchdf()

# Lukk tilkoblingen
con.close()


# Sjekk om vi fikk noen rader
if filtered_data.empty:
    print("Ingen data funnet i det spesifiserte tidsintervallet.")
else:
    print(f"Fant {len(filtered_data)} rader i tidsintervallet:")
    display(filtered_data)  # Viser hele den filtrerte tabellen

    # Sjekk om datasettet inneholder 'latitude' og 'longitude'
    if "latitude" in filtered_data.columns and "longitude" in filtered_data.columns:
        avg_lat = filtered_data["latitude"].mean()
        avg_lon = filtered_data["longitude"].mean()

        # Lag et Folium-kart med gjennomsnittlig senterpunkt
        m = folium.Map(location=[avg_lat, avg_lon], zoom_start=10)

        # Legg til punktene i kartet
        for _, row in filtered_data.iterrows():
            lat, lon = row["latitude"], row["longitude"]
            if not pd.isna(lat) and not pd.isna(lon):  # Sjekk at verdiene ikke er NaN
                ship_name = row.get("ship_name", "Ukjent skip")  # Erstatt med riktig kolonnenavn for skipets navn
                popup_text = f"Timestamp: {row['date_time_utc']}<br>Ship: {ship_name}"

                folium.Marker(
                    location=[lat, lon],
                    popup=popup_text,
                ).add_to(m)

        # Vis kartet
        display(m)
    else:
        print("Ingen 'latitude'/'longitude'-kolonner funnet i datasettet.")


Fant 15 rader i tidsintervallet:


,date_time_utc,mmsi,longitude,latitude,status,course_over_ground,speed_over_ground,rate_of_turn,maneuvre,imo,callsign,ship_name,ship_type,length,draught,data_source,ais_class,hex_7,hex_14,geometry
0,2024-12-01 05:12:51,257565700,7.476917,58.020340,5,134.2,0.0,16.0,0.0,0,LF5484,RESCUE 157 BILL,51,14,0.0,G,A,608154275335372799,639679472713910671,"[1, 1, 0, 0, 0, 95, 120, 70, 215, 92, 232, 29,..."
1,2024-12-01 05:13:58,258012180,8.034652,58.146895,15,360.0,0.0,-128.0,0.0,0,LF7245,SAR NICOLAI JARLSBY,51,9,0.6,G,A,608155171775578111,639680369155664551,"[1, 1, 0, 0, 0, 123, 231, 196, 220, 189, 17, 3..."
2,2024-12-01 05:13:47,258012180,8.034647,58.146887,15,360.0,0.0,-128.0,0.0,0,LF7245,SAR NICOLAI JARLSBY,51,9,0.6,G,A,608155171775578111,639680369155664783,"[1, 1, 0, 0, 0, 52, 59, 255, 52, 189, 17, 32, ..."
3,2024-12-01 05:13:38,258012180,8.034633,58.146860,15,360.0,0.0,-128.0,0.0,0,LF7245,SAR NICOLAI JARLSBY,51,9,0.6,G,A,608155171775578111,639680369155664791,"[1, 1, 0, 0, 0, 53, 103, 183, 117, 187, 17, 32..."
4,2024-12-01 05:13:28,258012180,8.034620,58.146840,15,360.0,0.0,-128.0,0.0,0,LF7245,SAR NICOLAI JARLSBY,51,9,0.6,G,A,608155171775578111,639680369155662183,"[1, 1, 0, 0, 0, 54, 147, 111, 182, 185, 17, 32..."
5,2024-12-01 05:13:18,258012180,8.034605,58.146835,15,360.0,0.0,-128.0,0.0,0,LF7245,SAR NICOLAI JARLSBY,51,9,0.6,G,A,608155171775578111,639680369155662183,"[1, 1, 0, 0, 0, 97, 142, 30, 191, 183, 17, 32,..."
6,2024-12-01 05:13:08,258012180,8.034602,58.146838,15,360.0,0.0,-128.0,0.0,0,LF7245,SAR NICOLAI JARLSBY,51,9,0.6,G,A,608155171775578111,639680369155662183,"[1, 1, 0, 0, 0, 180, 44, 12, 79, 183, 17, 32, ..."
7,2024-12-01 05:12:58,258012180,8.034608,58.146845,15,360.0,0.0,-128.0,0.0,0,LF7245,SAR NICOLAI JARLSBY,51,9,0.6,G,A,608155171775578111,639680369155662183,"[1, 1, 0, 0, 0, 209, 9, 219, 46, 184, 17, 32, ..."
8,2024-12-01 05:14:00,258258500,8.778695,58.461892,0,221.7,0.0,0.0,0.0,0,LDLN,RESCUE STORMBULL,51,17,1.0,G,A,608155183335079935,639680380713776863,"[1, 1, 0, 0, 0, 17, 30, 109, 28, 177, 142, 33,..."
9,2024-12-01 05:13:50,258258500,8.778688,58.461893,0,221.4,0.0,0.0,0.0,0,LDLN,RESCUE STORMBULL,51,17,1.0,G,A,608155183335079935,639680380713776863,"[1, 1, 0, 0, 0, 243, 64, 158, 60, 176, 142, 33..."


Display av ytelse (hastighet og filstørrelse)

Henter størrelse i Byte

In [16]:
os.path.getsize("/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/hais_2024-12-01.snappy.parquet.geo.parquet")

522358

Konverterer til MB

In [17]:
import os

# Filsti
filsti = "/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/hais_2024-12-01.snappy.parquet.geo.parquet"

# Hent filstørrelse i bytes
filstørrelse_bytes = os.path.getsize(filsti)

# Konverter til megabytes (MB)
filstørrelse_mb = filstørrelse_bytes / (1024 * 1024)

# Presenter resultatet
print(f"Filstørrelse: {filstørrelse_mb:.2f} MB")


Filstørrelse: 0.50 MB


under 1000 000 konverterer til KB, over 1000 000 konverterer til MB

In [19]:
import os

def sjekk_filstørrelse(filsti):
    """
    Sjekker størrelsen på en fil og konverterer den til enten megabytes (MB)
    hvis større enn 100 000 bytes, eller kilobytes (KB) hvis mindre.
    """
    # Hent filstørrelse i bytes
    filstørrelse_bytes = os.path.getsize(filsti)

    if filstørrelse_bytes > 1000_000:
        filstørrelse_mb = filstørrelse_bytes / (1024 * 1024)
        return f"Filstørrelse: {filstørrelse_mb:.2f} MB"
    else:
        filstørrelse_kb = filstørrelse_bytes / 1024
        return f"Filstørrelse: {filstørrelse_kb:.2f} KB"




Filstørrelse: 510.12 KB


In [18]:
import os

def sjekk_filstørrelse(filsti):
    """
    Returnerer størrelsen på en fil i MB eller KB.
    """
    if not os.path.exists(filsti):
        return "Filen finnes ikke."

    filstørrelse_bytes = os.path.getsize(filsti)

    if filstørrelse_bytes > 100_000:
        filstørrelse_mb = filstørrelse_bytes / (1024 * 1024)
        return f"{filstørrelse_mb:.2f} MB"
    else:
        filstørrelse_kb = filstørrelse_bytes / 1024
        return f"{filstørrelse_kb:.2f} KB"

def sammenlign_filstørrelse(original_fil, filtrert_fil):
    """
    Sammenligner filstørrelsen før og etter filtrering/partisjonering.
    """
    original_størrelse = sjekk_filstørrelse(original_fil)
    filtrert_størrelse = sjekk_filstørrelse(filtrert_fil)

    print(f"Original filstørrelse: {original_størrelse}")
    print(f"Filtrert/partisjonert filstørrelse: {filtrert_størrelse}")

# Eksempel på bruk:
original_fil = "/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/hais_2024-12-01.snappy.parquet.geo.parquet"
filtrert_fil = "/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/PartisjonertDataTimer/partitioned_1.parquet"

sammenlign_filstørrelse(original_fil, filtrert_fil)


Original filstørrelse: 0.50 MB
Filtrert/partisjonert filstørrelse: 86.09 KB


""Filtrering av de partisjonerte filene etter tid

In [22]:
# Opprett DuckDB-tilkobling
con = duckdb.connect()

# Definer tidsintervallet
start_time = "2024-12-01 11:12:51"
end_time = "2024-12-01 11:14:00"

# SQL-spørring for å filtrere data basert på tidspunkt
query = f"""
    SELECT *
    FROM read_parquet('/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/PartisjonertDataTimer/partitioned_11.parquet')
    WHERE CAST(date_time_utc AS TIMESTAMP)
    BETWEEN TIMESTAMP '{start_time}' AND TIMESTAMP '{end_time}'
"""

# Utfør spørringen og hent dataen
filtered_data = con.execute(query).fetchdf()

# Lukk tilkoblingen
con.close()


# Sjekk om vi fikk noen rader
if filtered_data.empty:
    print("Ingen data funnet i det spesifiserte tidsintervallet.")
else:
    print(f"Fant {len(filtered_data)} rader i tidsintervallet:")
    display(filtered_data)  # Viser hele den filtrerte tabellen

    # Sjekk om datasettet inneholder 'latitude' og 'longitude'
    if "latitude" in filtered_data.columns and "longitude" in filtered_data.columns:
        avg_lat = filtered_data["latitude"].mean()
        avg_lon = filtered_data["longitude"].mean()

        # Lag et Folium-kart med gjennomsnittlig senterpunkt
        m = folium.Map(location=[avg_lat, avg_lon], zoom_start=10)

        # Legg til punktene i kartet
        for _, row in filtered_data.iterrows():
            lat, lon = row["latitude"], row["longitude"]
            if not pd.isna(lat) and not pd.isna(lon):  # Sjekk at verdiene ikke er NaN
                ship_name = row.get("ship_name", "Ukjent skip")  # Erstatt med riktig kolonnenavn for skipets navn
                popup_text = f"Timestamp: {row['date_time_utc']}<br>Ship: {ship_name}"

                folium.Marker(
                    location=[lat, lon],
                    popup=popup_text,
                ).add_to(m)

        # Vis kartet
        display(m)
    else:
        print("Ingen 'latitude'/'longitude'-kolonner funnet i datasettet.")

Fant 13 rader i tidsintervallet:


,date_time_utc,mmsi,longitude,latitude,status,course_over_ground,speed_over_ground,rate_of_turn,maneuvre,imo,...,ship_type,length,draught,data_source,ais_class,hex_7,hex_14,geometry,hour,__index_level_0__
0,2024-12-01 11:13:58,258012180,8.034695,58.146992,15,360.0,0.0,-128.0,0.0,0,...,51,9,0.6,G,A,608155171775578111,639680369155664495,"[1, 1, 0, 0, 0, 96, 171, 4, 139, 195, 17, 32, ...",11,3969
1,2024-12-01 11:13:48,258012180,8.034695,58.146987,15,360.0,0.0,-128.0,0.0,0,...,51,9,0.6,G,A,608155171775578111,639680369155664455,"[1, 1, 0, 0, 0, 96, 171, 4, 139, 195, 17, 32, ...",11,3970
2,2024-12-01 11:13:39,258012180,8.034697,58.146982,15,360.0,0.0,-128.0,0.0,0,...,51,9,0.6,G,A,608155171775578111,639680369155664455,"[1, 1, 0, 0, 0, 251, 245, 183, 194, 195, 17, 3...",11,3971
3,2024-12-01 11:13:27,258012180,8.034702,58.146982,15,360.0,0.0,-128.0,0.0,0,...,51,9,0.6,G,A,608155171775578111,639680369155664455,"[1, 1, 0, 0, 0, 66, 162, 125, 106, 196, 17, 32...",11,3972
4,2024-12-01 11:13:19,258012180,8.034703,58.146980,15,360.0,0.0,-128.0,0.0,0,...,51,9,0.6,G,A,608155171775578111,639680369155664455,"[1, 1, 0, 0, 0, 24, 211, 134, 162, 196, 17, 32...",11,3973
5,2024-12-01 11:13:08,258012180,8.034703,58.146978,15,360.0,0.0,-128.0,0.0,0,...,51,9,0.6,G,A,608155171775578111,639680369155664455,"[1, 1, 0, 0, 0, 24, 211, 134, 162, 196, 17, 32...",11,3974
6,2024-12-01 11:12:58,258012180,8.034702,58.146975,15,360.0,0.0,-128.0,0.0,0,...,51,9,0.6,G,A,608155171775578111,639680369155664455,"[1, 1, 0, 0, 0, 66, 162, 125, 106, 196, 17, 32...",11,3975
7,2024-12-01 11:13:50,258258500,8.778725,58.461933,0,231.7,0.0,0.0,0.0,0,...,51,17,1.0,G,A,608155183335079935,639680380713654687,"[1, 1, 0, 0, 0, 187, 39, 15, 11, 181, 142, 33,...",11,15462
8,2024-12-01 11:13:41,258258500,8.778723,58.461930,0,233.9,0.0,0.0,0.0,0,...,51,17,1.0,G,A,608155183335079935,639680380713654679,"[1, 1, 0, 0, 0, 229, 246, 5, 211, 180, 142, 33...",11,15463
9,2024-12-01 11:13:30,258258500,8.778703,58.461888,0,232.4,0.1,0.0,0.0,0,...,51,17,1.0,G,A,608155183335079935,639680380713776863,"[1, 1, 0, 0, 0, 201, 69, 239, 51, 178, 142, 33...",11,15464


Nedlasting av filtrert fil


In [30]:


def sjekk_filstørrelse(filsti):
    """ Returnerer størrelsen på en fil i MB eller KB """
    if not os.path.exists(filsti):
        return "Fil ikke funnet"

    filstørrelse_bytes = os.path.getsize(filsti)
    if filstørrelse_bytes > 100_000:
        return f"{filstørrelse_bytes / (1024 * 1024):.2f} MB"
    else:
        return f"{filstørrelse_bytes / 1024:.2f} KB"

def sammenlign_filstørrelser(original_fil, partisjonert_fil, filtrert_fil):
    """ Sammenligner filstørrelsene før og etter filtrering/partisjonering """
    print(f"Original filstørrelse: {sjekk_filstørrelse(original_fil)}")
    print(f"Partisjonert filstørrelse: {sjekk_filstørrelse(partisjonert_fil)}")
    print(f"Filstørrelse etter filtrering: {sjekk_filstørrelse(filtrert_fil)}")

# Definer filstier
original_fil = "/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/hais_2024-12-01.snappy.parquet.geo.parquet"
partisjonert_fil = "/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/PartisjonertDataTimer/partitioned_11.parquet"
filtrert_fil = "/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/FiltrertData/filtered2.parquet"  # NY FIL

# Opprett DuckDB-tilkobling
con = duckdb.connect()

# Definer tidsintervallet
start_time = "2024-12-01 11:09:51"
end_time = "2024-12-01 11:14:00"

# SQL-spørring for å filtrere data basert på tidspunkt
query = f"""
    SELECT *
    FROM read_parquet('{partisjonert_fil}')
    WHERE CAST(date_time_utc AS TIMESTAMP)
    BETWEEN TIMESTAMP '{start_time}' AND TIMESTAMP '{end_time}'
"""

# Utfør spørringen og hent dataen
filtered_data = con.execute(query).fetchdf()

# Lagre det filtrerte datasettet som en ny Parquet-fil
if not filtered_data.empty:
    filtered_data.to_parquet(filtrert_fil)
    print(f"Filtrert data lagret som: {filtrert_fil}")

# Lukk tilkoblingen
con.close()

# Sammenlign filstørrelser
sammenlign_filstørrelser(original_fil, partisjonert_fil, filtrert_fil)

# Visualisering hvis datasettet har latitude/longitude
if not filtered_data.empty:
    print(f"Fant {len(filtered_data)} rader i tidsintervallet:")
    display(filtered_data)

    if "latitude" in filtered_data.columns and "longitude" in filtered_data.columns:
        avg_lat = filtered_data["latitude"].mean()
        avg_lon = filtered_data["longitude"].mean()
        m = folium.Map(location=[avg_lat, avg_lon], zoom_start=10)

        for _, row in filtered_data.iterrows():
            lat, lon = row["latitude"], row["longitude"]
            if not pd.isna(lat) and not pd.isna(lon):
                ship_name = row.get("ship_name", "Ukjent skip")
                popup_text = f"Timestamp: {row['date_time_utc']}<br>Ship: {ship_name}"

                folium.Marker(
                    location=[lat, lon],
                    popup=popup_text,
                ).add_to(m)

        display(m)
    else:
        print("Ingen 'latitude'/'longitude'-kolonner funnet i datasettet.")
else:
    print("Ingen data funnet i det spesifiserte tidsintervallet.")


Filtrert data lagret som: /Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/FiltrertData/filtered2.parquet
Original filstørrelse: 0.50 MB
Partisjonert filstørrelse: 49.40 KB
Filstørrelse etter filtrering: 15.37 KB
Fant 55 rader i tidsintervallet:


,date_time_utc,mmsi,longitude,latitude,status,course_over_ground,speed_over_ground,rate_of_turn,maneuvre,imo,...,ship_type,length,draught,data_source,ais_class,hex_7,hex_14,geometry,hour,__index_level_0__
0,2024-12-01 11:11:41,258012180,8.033333,58.146666,15,511.0,0.0,NaN,NaN,0,...,51,9,0.6,S,LRIT,608155171775578111,639680369155854543,"[1, 1, 0, 0, 0, 81, 193, 225, 5, 17, 17, 32, 6...",11,1040
1,2024-12-01 11:11:58,258258500,8.778333,58.461666,0,235.0,0.0,NaN,NaN,0,...,51,17,1.0,S,LRIT,608155183335079935,639680380713650551,"[1, 1, 0, 0, 0, 142, 203, 184, 169, 129, 142, ...",11,1059
2,2024-12-01 11:11:45,257004200,8.385055,58.246472,5,360.0,0.0,-128.0,0.0,0,...,51,12,0.0,G,A,608155174778699775,639680372160444071,"[1, 1, 0, 0, 0, 147, 82, 208, 237, 37, 197, 32...",11,1204
3,2024-12-01 11:11:06,257004200,8.385000,58.245000,5,511.0,0.0,NaN,NaN,0,...,51,12,0.0,S,LRIT,608155174812254207,639680372197824823,"[1, 1, 0, 0, 0, 133, 235, 81, 184, 30, 197, 32...",11,1511
4,2024-12-01 11:13:58,258012180,8.034695,58.146992,15,360.0,0.0,-128.0,0.0,0,...,51,9,0.6,G,A,608155171775578111,639680369155664495,"[1, 1, 0, 0, 0, 96, 171, 4, 139, 195, 17, 32, ...",11,3969
5,2024-12-01 11:13:48,258012180,8.034695,58.146987,15,360.0,0.0,-128.0,0.0,0,...,51,9,0.6,G,A,608155171775578111,639680369155664455,"[1, 1, 0, 0, 0, 96, 171, 4, 139, 195, 17, 32, ...",11,3970
6,2024-12-01 11:13:39,258012180,8.034697,58.146982,15,360.0,0.0,-128.0,0.0,0,...,51,9,0.6,G,A,608155171775578111,639680369155664455,"[1, 1, 0, 0, 0, 251, 245, 183, 194, 195, 17, 3...",11,3971
7,2024-12-01 11:13:27,258012180,8.034702,58.146982,15,360.0,0.0,-128.0,0.0,0,...,51,9,0.6,G,A,608155171775578111,639680369155664455,"[1, 1, 0, 0, 0, 66, 162, 125, 106, 196, 17, 32...",11,3972
8,2024-12-01 11:13:19,258012180,8.034703,58.146980,15,360.0,0.0,-128.0,0.0,0,...,51,9,0.6,G,A,608155171775578111,639680369155664455,"[1, 1, 0, 0, 0, 24, 211, 134, 162, 196, 17, 32...",11,3973
9,2024-12-01 11:13:08,258012180,8.034703,58.146978,15,360.0,0.0,-128.0,0.0,0,...,51,9,0.6,G,A,608155171775578111,639680369155664455,"[1, 1, 0, 0, 0, 24, 211, 134, 162, 196, 17, 32...",11,3974
